# Ex 1.2 Counting Cars
This task focuses on developing a computer vision application for counting the number of cars going from the city's downtown to the city centre in peak hours. It will be a progression to task 1.1 which uses the OpenCV library, and is based on frame differencing and background subtraction techniques. The project focuses on detecting cars that are in the "Main street".

In [1]:
import sys
import cv2
import numpy as np

The code will follow task 1.1's algorithm, tweaked to count cars.<br>The background subtractor MOG2 will be used with the same parameters as task 1.1.

In [2]:
# creating an instance of Gaussian Mixture-based background subtraction
background_sub = cv2.createBackgroundSubtractorMOG2( history = 500,
                                                     varThreshold = 50,
                                                     detectShadows = True )

# loading the video file
vid_path1, vid_path2 = "media/Traffic_Laramie_1.mp4", "media/Traffic_Laramie_2.mp4"
capture1 = cv2.VideoCapture(vid_path1)
capture2 = cv2.VideoCapture(vid_path2)

# checking if video 1 is opened successfully
if not capture1.isOpened(): print("Error: Could not open video 1.")
else: print("Successfully opened video 1.")
# checking if video 2 is opened successfully
if not capture2.isOpened(): print("Error: Could not open video 2.")
else: print("Successfully opened video 2.")

Successfully opened video 1.
Successfully opened video 2.


The algorithm detects and counts vehicles moving towards the city center. The Region of Interest (ROI) is applied to focus detection on the main street. ```cv2.dilate()``` and ```cv2.erode()``` are used to ensure that the detected vehicles are not fragmented whilst removing noise.

Moving objects are then detected, non-vehicle objects are filtered out based on the aspect ratio and contour area. Vehicles are tracked using their center coordinates to ensure that they are counted only once when crossing into the city center.

In [3]:
# defining main street region of interest (ROI)
roi_top_left = (0, 250)
roi_bot_right = (1040, 600)

# defining entry and exit line
entry_y = 430 # entering towards city centre
exit_y = 350 # crossing into city centre

# storing tracked cars
tracked_cars = {}
car_count = 0

capture = capture1

while True:
    check, img = capture.read()
    if not check: break # stop if end of video
        
    # applying background subtraction
    fg_mask = background_sub.apply(img)
    # dilating white areas of detected object
    fg_mask = cv2.dilate(fg_mask, None, 20)
    # shrinking white areas in binary mask
    fg_mask = cv2.erode(fg_mask, None, 15)

    # applying thresholding to clean up noise
    threshold_mask = cv2.threshold(fg_mask, 150, 255, cv2.THRESH_BINARY)[1]
    
    # applying ROI (main street)
    roi = np.zeros_like(threshold_mask)
    cv2.rectangle(roi, roi_top_left, roi_bot_right, 255, thickness = -1)
    threshold_mask = cv2.bitwise_and(threshold_mask, roi)
    
    # finding contours to detect moving objects
    contours = cv2.findContours(threshold_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    # storing bounding boxes of cars
    curr_cars = []
    
    for c in contours:
        # drawing bounding box around detected cars
        x, y, w, h = cv2.boundingRect(c)
        # filtering out human shapes or small contours
        if (h/w > 1.3) | (cv2.contourArea(c) < 1100): continue
            
        # getting car's center
        center_x = x + w // 2
        center_y = y + h // 2
        
        curr_cars.append((center_x, center_y, x, y, x + w, y + h))
    
    # tracking movement of cars across frames
    new_tracked_cars = {}
    
    for car in curr_cars:
        center_x, center_y, x1, y1, x2, y2 = car
        
        matched = False
        for car_id, (prev_x, prev_y) in tracked_cars.items():
            # checking for distance change
            if abs(center_x - prev_x) < 50 and abs(center_y - prev_y) < 50 and abs(center_x - prev_x) > 5:
                new_tracked_cars[car_id] = (center_x, center_y)
                
                # counting cars if it crosses from entry to exit
                if prev_y >= entry_y and center_y < entry_y:
                    car_count += 1
                    print(f"Car detected, Total: {car_count}")
                    
                matched = True
                break
                
        # assigning new ID to newly detected cars
        if not matched:
            new_tracked_cars[len(tracked_cars) + 1] = (center_x, center_y)
    
    tracked_cars = new_tracked_cars # updating car tracking
    # draw entry and exit lines
    cv2.line(img, (0, entry_y), (img.shape[1], entry_y), (255, 0, 0), 2)
    cv2.line(img, (0, exit_y), (img.shape[1], exit_y), (0, 0, 255), 2)
    
    # drawing bounding box on detected cars
    for car in curr_cars:
        _, _, x1, y1, x2, y2 = car
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
    # displaying car count
    cv2.putText(img, f"Cars counted: {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    # displaying the results
    cv2.imshow("Traffic Video", img)
    
    # exiting the loop
    key = cv2.waitKey(1)
    # using "q" key to exit
    if key == ord("q"):
        break

# after the loop, release the video object
capture.release()
# destroy all windows
cv2.destroyAllWindows()

Car detected, Total: 1
Car detected, Total: 2
Car detected, Total: 3
Car detected, Total: 4
Car detected, Total: 5
Car detected, Total: 6
